# **Minggu 5: Feature Detection**


(*Tutorial Rujukan: [Dokumentasi OpenCV untuk Feature Detection](https://docs.opencv.org/3.4/db/d27/tutorial_py_table_of_contents_feature2d.html)*)


Pada bagian ini kita akan menggunakan teknik deteksi fitur dan matching untuk melakukan beberapa pengolahan citra dan memperoleh informasi. Ingat bahwa pengolahan citra perlu dilakukan secara bertahap dengan teknik yang boleh jadi berbeda pada kondisi yang berbeda. Untuk itu ingat kembali pelajaran pada minggu-minggu sebelumnya sebelum melanjutkan pada materi di Minggu ini.

Contoh gambar untuk latihan pada minggu ini tersedia pada eLOK mata kuliah PCD. Unduh tiap gambar, kemudian unggah pada Google Colab menggunakan cara yang telah diberikan.

## Review: Ekstraksi objek pada citra

Pada saat melihat sebuah foto atau gambar digital, manusia dapat dengan mudah membedakan objek yang digambarkan pada foto tersebut. Sebagai contoh, dari gambar berikut:

![alt text](https://drive.google.com/uc?id=1kc8QA4GIHqaQusFlcGd91cUNnb3nr7GU)

Manusia dapat dengan mudah membedakan jeruk dan daun, jeruk yang dekat dengan yang jauh, serta menghitung jumlah jeruk yang ada, misalnya.

Untuk komputer, hal ini menjadi kompleks karena komputer hanya melihat serangkaian pixel, tanpa kemampuan untuk memisahkan fitur tersebut. 

Pada [pertemuan sebelumnya](https://colab.research.google.com/drive/1To2FZnLbxdhRSPW3lZlUQymaV2T5lF_m), telah dibahas mengenai cara 'mengajarkan' komputer agar mampu melihat dan membedakan warna (Hue) sehingga dapat digunakan untuk ekstraksi fitur. Pada [minggu selanjutnya](https://colab.research.google.com/drive/1FpvJQnGb3_A85oktAXJOzGYG24DK26u1) juga telah dibahas mengenai metode filtering dan thresholding yang dapat digunakan untuk menonjolkan **tepi** sebuah objek sehingga dapat dilakukan ekstraksi fitur, misalnya delineasi garis pantai.

![alt text](https://slideplayer.com/slide/14894726/91/images/6/Images+as+functions%E2%80%A6+Edges+look+like+steep+cliffs.jpg)

Dengan menggunakan nilai piksel pada sebuah citra digital, kita dapat memberikan informasi pada komputer agar dapat mengenali suatu objek. Namun bagaimana dengan kedalaman objek tersebut? Pembahasan minggu ini merupakan pengantar untuk materi *depth reconstruction* melalui pengolahan citra digital.





## Apa itu Fitur?

'Fitur' merupakan bagian dari citra yang dapat diidentifikasi dengan mudah oleh komputer. Sebagai contoh, pada gambar berikut: 

![alt text](https://drive.google.com/uc?id=1lDujYsNKdilG_d-Uv4gt0hzoPzBKDZ6q)

Manakah yang merupakan 'fitur' yang mudah untuk diidentifikasi?

Jawabnya adalah ujung persegi panjang yang ditandai oleh kotak berwarna merah. Kedua fitur lain cukup sulit untuk diidentifikasi:
*   Kotak warna biru dapat berada pada posisi manapun di dalam persegi panjang warna hijau
*   kotak warna hitam dapat berada dimana saja selama berada pada tepian persegi panjang.

Dengan demikian, sebuah citra dapat dilatih untuk mencari titik-titik yang mudah dilacak (*Good Features To Track*) dengan membuat kategori untuk tiap piksel dengan tetangga piksel masing-masing. 

Terdapat beberapa algoritma untuk melakukan deteksi pojok ('Corner Detector') dari sebuah citra digital. Beberapa yang cukup populer adalah:

*   [Harris Corner Detector](https://docs.opencv.org/3.4/dc/d0d/tutorial_py_features_harris.html)
*   [Shi-Tomasi dan Good Features to Track](https://docs.opencv.org/3.4/d4/d8c/tutorial_py_shi_tomasi.html)

Lakukan latihan berikut untuk memahami mengenai Good Features to Track pada sebuah Citra Digital






### Latihan 1: Menggunakan Corner Detector

Lakukan latihan berikut menggunakan kedua metode yang tersedia pada OpenCV (Harris Corner dan Shi-Tomasi). Gunakan gambar yang berbeda dan bandingkan jumlah corner yang berhasil dideteksi oleh kedua algoritma tersebut.

In [1]:
!wget https://pns2019.github.io/images/Lenna.png

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# Menggunakan Shi-Tomasi GFTT untuk deteksi ujung (corner detection)

import numpy as np
import cv2
from matplotlib import pyplot as plt

# gunakan gambar yang disediakan di eLOK
img = cv2.imread('gedungpusat.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 


# deteksi pojok dengan GFTT
corners = cv2.goodFeaturesToTrack(gray,1000,0.01,10)
corners = np.int0(corners)

# menampilkan jumlah titik terdeteksi dengan fungsi numpy (np.ndarray.shape)
print("jumlah titik terdeteksi = ", corners.shape[0])

# untuk ditampilkan di Matplotlib, urutan band dibalik
rgb = cv2.cvtColor(img,cv.COLOR_BGR2RGB)

# perbesar ukuran hasil plotting 
plt.rcParams["figure.figsize"] = (20,20)

# untuk tiap pojok yang terdeteksi, munculkan pada gambar
for i in corners:
    x,y = i.ravel()
    cv2.circle(rgb,(x,y),3,255,-1)
plt.imshow(rgb),plt.show()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


## Deteksi Keypoints (Feature Detection)

*(Rujukan: http://www.fossreview.com/2018/06/studying-digital-image-with-opencv4.html)*


---


Deteksi ujung (corner detection) dapat digunakan untuk 
mengidentifikasi fitur pada satu foto dengan baik. Namun demikian, metode ini kurang dapat digunakan dengan baik apabila diperlukan untuk mengidentifikasi titik yang sama pada foto yang berbeda. Kebutuhan ini akan kita jumpai pada saat kita mencari hubungan geometri pada dua buah foto, misalnya (ingat materi [minggu ke-4](https://colab.research.google.com/drive/1ZmddpA9v5dnzKJ8Up_So0AfjKjroOojZ)).

Sebagai contoh, pada gambar berikut:

![alt text](https://drive.google.com/uc?id=1c3g2BTFgSunu2BXqKLViVkr6u3pjT8Of)

Corner detection tidak akan banyak berguna untuk mencari titik-titik yang sama dan berpasangan pada kedua buah gambar (bayangkan '*tie-points*' pada fotogrametri). Hal ini disebabkan karena gambar kedua memiliki posisi yang berbeda (mengalami rotasi, translasi dan/atau skala), sehingga deteksi ujung tidak dapat digunakan. 

Gambar berikut menunjukkan kelemahan corner detector apabila digunakan pada gambar dengan perubahan skala (misalnya beda tinggi terbang foto udara)

![alt text](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/sift_scale_invariant.jpg)

Pada pelajaran minggu sebelumnya telah diberikan contoh bahwa dari beberapa **titik sekutu** pada dua buah foto dapat dicari matriks, atau **HOMOGRAPHY** yang menyatakan hubungan antara kedua sistem foto tersebut. Dengan demikian, permasalahannya adalah bagaimana mencari titik-titik yang dapat dideteksi pada kedua buah foto sekaligus dapat dihubungkan antara titik satu dengan yang lain?

Untuk itu, dikembangkan suatu metode untuk melakukan deteksi *keypoint* pada tiap foto. **Keypoint** dapat disebut sebagai sebuah 'basisdata' untuk tiap titik dengan karakteristik tertentu berdasarkan atas nilai piksel tetangganya. Dengan membentuk basisdata titik yang cukup detil, dua buah titik dapat saling dipasangkan apabila memiliki kemiripan yang cukup. Dengan demikian, matriks homography dapat dihitung dan rekonstruksi posisi foto dapat dilakukan sebagaimana materi yang telah disampaikan sebelumnya.

Pada OpenCV, terdapat beberapa fungsi deteksi keypoints, antara lain:

*   [SIFT (Scale-Invariant Feature Transform)](https://docs.opencv.org/3.4/da/df5/tutorial_py_sift_intro.html). Merupakan metode yang paling terkenal dan banyak digunakan
*   [SURF (Speeded-Up Robust Feature)](https://docs.opencv.org/3.4/df/dd2/tutorial_py_surf_intro.html).
* [FAST](https://docs.opencv.org/3.4/df/d0c/tutorial_py_fast.html) 
* [BRIEF (Binary Robust Independent Elementary Features)](https://docs.opencv.org/3.4/dc/d7d/tutorial_py_brief.html). Merupakan descriptor yang digunakan untuk menghitung hasil deteksi secara lebih efisien. BRIEF seringkali dipasangkan dengan FAST.
* [ORB (Oriented FAST and Rotated BRIEF)](https://docs.opencv.org/3.4/d1/d89/tutorial_py_orb.html), merupakan alternatif gratis untuk SIFT dan SURF (keduanya memiliki paten)
* [A-KAZE](https://docs.opencv.org/3.4/db/d70/tutorial_akaze_matching.html). Salah satu feature detector terbaru yang banyak digunakan di software SfM Opensource


Fungsi-fungsi di atas dapat dipanggil dengan menggunakan fungsi features2d pada OpenCV sebagai berikut:

```
sift = cv2.xfeatures2d.SIFT_create()
surf = cv2.xfeatures2d.SURF_create()
orb = cv2.ORB_create(nfeatures=1500)

# FAST dan BRIEF
# FAST detector
star = cv.xfeatures2d.StarDetector_create()
# BRIEF extractor
brief = cv.xfeatures2d.BriefDescriptorExtractor_create()
```

Karena algoritma SIFT dan SURF dipatenkan dan berbayar sedangkan OpenCV adalah perangkat lunak opensource, mulai OpenCV versi 4 semua fungsi tersebut dipisahkan dari modul utama OpenCV. Dengan demikian, untuk menggunakan fungsi tersebut dapat digunakan OpenCV versi lama (Versi 3) yang masih mengandung modul SIFT dan SURF.

### Latihan 2: Menggunakan ORB dan KAZE

ORB dan KAZE merupakan contoh feature detector yang bersifat *opensource*, sehingga dapat digunakan secara gratis pada OpenCV. 

Lakukan latihan berikut dengan gambar yang berbeda. Apabila script menggunakan gambar grayscale, tampilkan gambar berwarna dengan menggunakan Matplotlib.

In [ ]:
# Mendownload gambar Lenna untuk contoh
#!wget https://pns2019.github.io/images/Lenna.png

import cv2
import numpy as np

# memanggil gambar berwarna
img = cv2.imread('gedungpusat.jpg')

# cara lain memanggil grayscale dari gambar
#img = cv2.imread('gedungpusat.jpg',0)
gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# mendefinisikan KAZE descriptor
kaze = cv2.KAZE_create()
kp = kaze.detect(gray)

#kps = sorted(kp, key=lambda x: -x.response)[:32]

# computing descriptors vector
kp, dsc = kaze.compute(gray, kp)

# berapa titik yang terdeteksi?
print("jumlah titik terdeteksi = ", len(kp))

# menggambar keypoint yang berhasil diidentifikasi
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img2 = cv2.drawKeypoints(img_rgb, kp, None, color=(0,255,0), flags=0)
plt.imshow(img2), plt.show()

Contoh di atas menggunakan KAZE untuk mendeteksi titik. Ingat bahwa **keypoint** yang dihasilkan di proses ini berbeda dengan **corner** yang kita peroleh pada saat menggunakan corner detector seperti Harris di atas. Keypoint merupakan basisdata titik yang juga mengandung informasi mengenai piksel di sekitarnya sehingga kita dapat memasangkan tiap keypoint ini meskipun dideteksi pada foto yang berbeda.

Contoh di bawah menggunakan ORB untuk mendeteksi keypoint. Apa yang dapat Anda amati?

In [ ]:
# Latihan ORB

import numpy as np
import cv2 
from matplotlib import pyplot as plt

# membaca citra
img = cv2.imread('gedungpusat.jpg',0)

# membuat ORB detector
orb = cv2.ORB_create()

# kp: variabel untuk menyimpan keypoint yang berhasil dideteksi
kp = orb.detect(img,None)

# menghitung deskriptor
# kp = keypoints
# des = descriptor
kp, des = orb.compute(img, kp)

# berapa jumlah titik terdeteksi?
print("jumlah titik terdeteksi = ", len(kp))

# menggambar keypoint yang berhasil diidentifikasi 
img2 = cv2.drawKeypoints(img_rgb, kp, None, color=(0,255,0), flags=0)  # img_rbg dari variabel sebelumnya di atas
plt.imshow(img2), plt.show()


### Latihan 3: Menggunakan SIFT dan SURF

Berbeda dengan ORB dan KAZE (dan beberapa algoritma keypoint-detector lain) yang diberikan dalam bentuk open source, SIFT dan SURF merupakan algoritma yang telah dipatenkan, sehingga dihapus pada OpenCV versi 4 ke atas dan dirubah menjadi *community module*. 

Untuk keperluan praktek ini, untuk menggunakan modul SIFT dan SURF, terlebih dahulu kita harus uninstall OpenCV bawaan dari Google Colab, kemudian melakukan instalasi versi OpenCV yang lebih rendah. 

In [ ]:
# untuk menjalankan fungsi detektor SIFT, SURF dan ORB pada OpenCV,
# terlebih dahulu uninstall OpenCV versi 4 yang secara default terinstall
# pada Google Colab, kemudian lakukan kembali instalasi versi OpenCV 
# yang lebih rendah sehingga fungsi SIFT dkk dapat digunakan

!pip uninstall opencv-python -y

# Downgrade versi OpenCV menjadi versi 3
!pip install opencv-contrib-python==3.4.2.17 --force-reinstall


Setelah melakukan instalasi seperti di atas, kita harus melakukan restart pada Runtime Google Colab agar versi OpenCV yang baru (versi 3) dapat kita gunakan. Setelah restart, lakukan pengecekan versi opencv terinstall seperti berikut:


In [ ]:
# cek versi opencv sekarang 
import cv2
print(cv2.__version__)

Setelah OpenCV versi 3 berhasil diinstal, barulah fungsi berikut dapat dijalankan:

In [ ]:
# Melakukan deteksi keypoint dengan algoritma SIFT

import cv2
from matplotlib import pyplot as plt

img = cv2.imread('gedungpusat.jpg')
# konversi ke warna abu2 agar menjadi satu band
gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# menghitung fitur dengan SIFT
sift = cv2.xfeatures2d.SIFT_create()
kp = sift.detect(gray, None)

# berapa jumlah titik terdeteksi?
print("jumlah titik terdeteksi= ", len(kp))

# perbesar ukuran hasil plotting 
plt.rcParams["figure.figsize"] = (20,20)

# menggambar keypoint yang berhasil diidentifikasi 
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img3 =cv2.drawKeypoints(img_rgb,kp,img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(img3), plt.show()

Berbeda dengan ORB dan KAZE di atas, SIFT menggunakan definisi keypoint yang terlihat lebih beragam: selain posisi dan piksel di sekeliling titik, SIFT juga mengidentifikasi *orientasi* dari tiap keypoint, sehingga memungkinkan fitur yang ter-rotasi (misalnya foto udara) untuk dapat dipasangkan satu dengan yang lain.

Berdasarkan contoh-contoh di atas, coba buat detektor keypoint dengan menggunakan SURF. Perintah untuk SURF adalah seperti berikut:

```
# definisi SURF
surf = cv2.SURF(400)

# mencari dan menghitung keypoint
kp, des = surf.detectAndCompute(img,None)

```

Bagaimana perbandingan jumlah kypoint yang dideteksi berdasarkan semua algoritma yang telah disebutkan di atas?

## Feature Matching

Rujukan: https://pysource.com/2018/03/23/feature-matching-brute-force-opencv-3-4-with-python-3-tutorial-26/


Apabila feature detector digunakan untuk mencari titik yang dapat dipasangkan, maka **Feature Matching** digunakan untuk memasangkan masing-masing keypoint yang sudah terdeteksi pada satu citra dengan titik titik yang memiliki kesamaan karakter pada citra lain. Dengan demikian, dapat dikatakan bahwa hasil dari feature matching adalah **Tie-Points** pada sebuah proses fotogrametri.

![alt text](https://docs.opencv.org/3.4/matcher_flann.jpg)

Pada Gambar di atas, lingkaran berwarna merah merupakan fitur (keypoints) yang dideteksi dengan menggunakan *Feature Detector* pada OpenCV, sedangkan garis-garis berwarna hijau merupakan titik-titik yang berhasil dipasangkan berdasarkan atas kesamaan karakteristik (*feature matching*).

Terdapat beberapa algoritma yang berbeda pula pada metode Feature Matching, antara lain:


*   Brute-Force Matching
*   FLANN

(Panduan untuk keduanya dapat dilihat [di sini](https://docs.opencv.org/3.4/dc/dc3/tutorial_py_matcher.html))

Penggunaan kedua model feature matching tersebut perlu disesuaikan dengan algoritma keypoint detector yang digunakan.



### Latihan 4: Feature Detection and Matching

Setelah memperoleh 'basisdata' keypoint, langkah selanjutnya adalah memasangkan masing-masing titik keypoint tersebut pada pasangannya yang memiliki kesamaan karakteristik. Contoh di bawah adalah bagaimana Feature Matching digunakan untuk memasangkan titik-titik keypoints yang berhasil ditemukan:

In [ ]:
# Contoh Script untuk feature detection and Matching
# Modifikasi script ini untuk mencoba metode yang berbeda 

import numpy as np
import cv2
from matplotlib import pyplot as plt

# Gunakan gambar dari eLOK
img1 = cv2.imread('webarebears.jpg')          # gambar yang dituju
img2 = cv2.imread('ice_bear.jpg')             # gambar yang dicari
gray1= cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
gray2= cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

# Menggunakan Detector SIFT
sift = cv2.xfeatures2d.SIFT_create()

# Mencari Keypoint dengan SIFT
kp1, des1 = sift.detectAndCompute(gray1,None)
kp2, des2 = sift.detectAndCompute(gray2,None)

# Melakukan Matching dari hasil deteksi keypoints menggunakan
# BruteForce Matcher
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

# Uji rasio matching sederhana
good = []
for m,n in matches:
    if m.distance < 0.5*n.distance:
        good.append([m])
img3 = None

# menggambar hasil match pada gambar baru (IMG3)
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,img3,flags=2)
plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)),plt.show()

Dari contoh di atas dapat dilihat bahwa dengan metode deteksi keypoint menggunakan SIFT kita dapat melakukan matching pada objek yang mengalami rotasi (ingat bahwa *SIFT=Scale-Invariant Feature Transform*). Aplikasi dari algoritma ini untuk bidang fotogrametri adalah pada deteksi titik tie-points secara otomatis pada dua buah foto yang bertampalan, sedangkan pada bidang remote sensing, metode ini digunakan dalam pembuatan mosaik otomatis untuk mendukung metode registrasi citra, semisal dengan RPC.

## Template Matching

Dari pelajaran di atas serta contoh-contoh praktikum beberapa minggu sebelumnya, kita telah melihat bagaimana algoritma level rendah seperti transformasi colorspace, edge detection, feature detection dan matching dapat sangat berguna untuk memperoleh berbagai informasi yang kita inginkan pada sebuah citra. Dengan memahami cara kerja dari masing-masing algoritma tersebut kita akan lebih mudah mengetahui bagaimana perangkat lunak pengolahan citra bekerja di belakang layar serta bagaimana melakukan pengaturan parameter yang dapat digunakan untuk mengoptimalkan hasil pemrosesan citra yang kita inginkan. 

Pada bagian ini kita akan menggunakan salah satu fungsi OpenCV yang digunakan untuk meringkas beberapa langkah di atas, yaitu deteksi keypoint, matching dan deteksi tepi untuk mendeteksi posisi wajah dari serangkaian gambar. Algoritma ini disebut sebagai 'Template Matching'.  OpenCV menyediakan fungsi `cv2.matchTemplate()` untuk keperluan Template Matching ini dengan berbagai metode deteksi berdasarkan tingkat kemiripan antara `template` atau gambar yang dicari dengan objek pada gambar yang tersedia , misalnya dengan menghitung rerata korelasi antar nilai piksel pada template tersebut dengan objek yang dicari. Latihan berikut menunjukkan aplikasi dari berbagai metode tersebut untuk mencari gambar yang diberikan.

Kita panggil terlebih dahulu kedua gambar:

In [ ]:
# tampilkan kedua gambar
from matplotlib import pyplot as plt

# panggil dan konversi warna agar sesuai dengan Matplotlib
einstein = cv2.imread('einstein.png')
einstein =  cv2.cvtColor(einstein, cv2.COLOR_BGR2RGB) # simpan dengan nama yang sama = ditumpuk

# panggil dan konversi warna agar sesuai dengan Matplotlib
solvay = cv2.imread('solvayconference.jpg')
solvay =  cv2.cvtColor(solvay, cv2.COLOR_BGR2RGB) 

plt.subplot(121),plt.imshow(einstein), plt.title('Einstein')
plt.subplot(122),plt.imshow(solvay), plt.title('Solvay Conference 1927')
plt.show()

Selanjutnya, lakukan Template Matching pada gambar Einstein dan Gambar Solvay Conference untuk mencari di mana Einstein duduk:

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('solvayconference.jpg',0)
img2 = img.copy()
template = cv2.imread('einstein.png',0)
w, h = template.shape[::-1]

# All the 6 methods for comparison in a list
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
            'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']

# perbesar ukuran hasil plotting 
plt.rcParams["figure.figsize"] = (15,15)

for met in methods:
    img = img2.copy()
    method = eval(met)

    # menggunakan template matching
    res = cv2.matchTemplate(img,template,method)

    # mencari ukuran citra template untuk menggambar kotak
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    # metode TM_SQDIFF dan TM_SQDIFF_NORMED menggunakan persamaan yang sedikit berbeda
    # sehingga dibuatkan fungsi khusus untuk mengambil nilai minimum
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)

    
    
    # buat persegi pada lokasi yang ditemukan
    cv2.rectangle(img, top_left, bottom_right, 255, 2) # 2 adalah ketebalan garis kotak

    print("hasil metode", met, ": " )
    plt.subplot(121),plt.imshow(res,cmap = 'gray')
    plt.title('Hasil matching'), plt.xticks([]), plt.yticks([])
    plt.subplot(122),plt.imshow(img,cmap = 'gray')
    plt.title('Lokasi terdeteksi'), plt.xticks([]), plt.yticks([])
    

    plt.show()

Pada contoh di atas, beberapa metode matching memberikan hasil yang lebih baik dibanding yang lain. Untuk tiap kondisi yang berbeda boleh jadi penggunaan parameter yang berbeda akan memberikan hasil yang lebih baik. Demikian pula, preprocessing (seperti melakukan deteksi tepi) biasanya akan membantu dalam proses matching seperti ini.

> Apa yang terjadi jika kita menggunakan foto Einstein yang lain? apakah program tetap dapat mendeteksi posisinya?

Pada bidang remote sensing, aplikasi dari Template Matching ini antara lain adalah untuk menghitung jumlah objek. Jika pada pelajaran di minggu sebelumnya kita menggunakan metode thresholding dan contouring untuk menghitung jumlah objek yang dapat ditemukan berdasarkan deteksi tepi, maka pada bagian ini kita akan menggunakan teknik template matching berdasarkan deteksi keypoint untuk mendeteksi jumlah objek. 

Kita akan mencoba melakukan deteksi jumlah pohon sawit dengan algoritma sederhana di atas. Gunakan file `sawit.png` sebagai template dan `plantation.jpg` pada eLOK UGM sebagai gambar yang akan kita cari dan hitung jumlahnya. Unggah kedua file tersebut pada Google Colab seperti sebelumnya.

In [ ]:
# tampilkan kedua gambar
from matplotlib import pyplot as plt

# panggil dan konversi warna agar sesuai dengan Matplotlib
sawit = cv2.imread('sawit.png')
sawit =  cv2.cvtColor(sawit, cv2.COLOR_BGR2RGB)

# panggil dan konversi warna agar sesuai dengan Matplotlib
kebun_sawit = cv2.imread('plantation.jpg')
kebun_sawit =  cv2.cvtColor(kebun_sawit, cv2.COLOR_BGR2RGB) 

plt.subplot(121),plt.imshow(sawit), plt.title('sawit')
plt.subplot(122),plt.imshow(kebun_sawit), plt.title('kebun sawit')
plt.show()


Pada contoh script di bawah, kita akan menggunakan template matching untuk menentukan jumlah pohon sawit berdasarkan template sawit seperti di atas. Parameter threshold di bawah menentukan seberapa mirip keypoint yang ingin kita deteksi berdasarkan template, sehingga kita dapat merubah nilai tersebut untuk mendapatkan jumlah yang lebih optimal.

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

## membaca gambar utuh untuk dicari
img_rgb = cv2.imread('plantation.jpg')
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

## membaca template 
template = cv2.imread('sawit.png',0)

## ukuran template. ukuran ini akan digunakan untuk menggambar kotak
w, h = template.shape[::-1]

# menggunakan metode COEFF-NORMALIZED
res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)

# Nilai threshold atau ambang batas deteksi kemiripan titik. 
# Lakukan eksperimen dengan merubah nilai ini
threshold = 0.15
loc = np.where(res >= threshold)

## membuat array kosong untuk menyimpan lokasi-lokasi dari hasil deteksi
lspoint=[]
lspoint2=[]
count = 0  # untuk menyimpan jumlah matching yang ditemukan
for pt in zip(*loc[::-1]):
	## jika sudah ada, skip lokasi tersebut
	if pt[0] not in lspoint and pt[1] not in lspoint2:
		## gambar persegi warna kuning dengan ketebalan dua poin
		cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,255,255), 2)
		for i in range(((pt[0])-9), ((pt[0])+9),1):
			## tambahkan koordinat x ke list
			lspoint.append(i)
		for k in range(((pt[1])-9), ((pt[1])+9),1):
			## tambahkan koordinat y ke list
			lspoint2.append(k)
		count+=1 ### berapa jumlah matching yang ditemukan?
	else:
		continue
print ("Jumlah objek ditemukan ", count)

## tampilkan dengan imshow	
cv2_imshow(img_rgb)

Hasil dari deteksi titik menunjukkan bahwa deteksi hanya dengan mengandalkan keypoint semata ternyata tidak berhasil dengan baik. Selain bahwa tidak semua titik terdeteksi, ternyata hasil deteksi juga melenceng dari posisi yang seharusnya. Ini adalah pertanyaan untuk Anda. 

*Mengapa ini terjadi?*

*Bagaimana cara agar hasilnya lebih baik?*

*Kira-kira untuk aplikasi apakah metode ini tepat untuk digunakan?*

Jawaban untuk pertanyaan-pertanyaan tersebut akan kita bahas sampai di akhir pertemuan Mata Kuliah Pengolahan Citra Digital ini.

## Resume

Feature Detector dan Feature Matching merupakan metode untuk 1) mencari dan membuat basisdata titik untuk dipasangkan, dan 2) memasangkan tiap titik berdasarkan kesamaan karakteristik tertentu. Dalam fotogrametri, kedua metode ini berguna untuk mencari **tie points** secara otomatis pada dua buah gambar stereo. Selanjutnya, setelah titik-titik dapat diidentifikasi dan dipasangkan, dapat direkonstruksi geometri pengambilan gambar kedua foto tersebut meskipun *tanpa menggunakan GCP atau posisi awal kamera*. Hal ini akan merupakan pembahasan awal mengenai SfM (Structure from Motion). Untuk lebih jelasnya, silahkan mencari sumber-sumber lain yang terkait dengan bagaimana SfM bekerja untuk melakukan rekonstruksi geometri 3D dari serangkaian foto udara yang bertampalan.

![](https://raw.githubusercontent.com/magicleap/SuperGluePretrainedNetwork/master/assets/freiburg_matches.gif)


Rangkuman algoritma yang tersedia pada OpenCV dapat dilihat pada gambar berikut:

![alt text](https://drive.google.com/uc?id=1DnWKbvF5rOqXnRQhIn6Qp-aqcjsni_ik)

Materi ini merupakan dasar dari bagaimana operasi citra pada level rendah digunakan untuk berbagai keperluan lain yang banyak dijumpai pada bidang penginderaan jauh. Untuk lebih memahamkan mengenai materi pada minggu ini, kerjakan latihan-latihan di bawah.

## Latihan Soal

1.   Gunakan gambar bebas (bisa foto/cari di internet) dan lakukan **deteksi corner** menggunakan 1) metode Harris dan 2) GFTT. Bandingkan hasil kedua metode tersebut. (Latihan 1)
Mana yang lebih baik menurut anda? 
Mengapa? 
Berapa jumlah corner yang berhasil dideteksi oleh masing-masing metode?
2.   Gunakan gambar bebas (bisa foto/cari di internet) dan lakukan **deteksi keypoint** menggunakan: SIFT, SURF, BRIEF dan ORB. (Latihan 2 dan 3)
Mana yang lebih baik menurut anda? 
Mengapa? 
Berapa jumlah keypoint yang berhasil dideteksi oleh masing-masing metode?
3.   Gunakan gambar bebas (misalnya **wajah masing-masing** , bisa dengan foto/selfie) untuk mendemokan fungsi Feature Detection and Template Matching pada OpenCV. Carilah kondisi optimal dimana teknik tersebut dapat digunakan dengan baik untuk mendeteksi jumlah objek dari foto udara atau citra satelit.





## Rujukan

* https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_features_harris/py_features_harris.html
* https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_matcher/py_matcher.html
* http://www.acgeospatial.co.uk/template-matching-eo/
* https://www.geeksforgeeks.org/template-matching-using-opencv-in-python/ 
* https://www.pyimagesearch.com/2018/12/17/image-stitching-with-opencv-and-python/